In [ ]:
!pip install llama-index

In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import FaithfulnessEvaluator


In [7]:
# create llm
llm = OpenAI(model="gpt-4", temperature=0.0)


In [9]:
documents = SimpleDirectoryReader("/content").load_data()

In [12]:
index = VectorStoreIndex.from_documents(documents)

In [14]:
# query index
query_engine = index.as_query_engine()
response = query_engine.query(
    "What are the Key Sectors of the Indian Economy?"
)

In [15]:
# define evaluator
evaluator = FaithfulnessEvaluator(llm=llm)

In [17]:
!pip install nest_asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()
eval_result = evaluator.evaluate_response(response=response)

In [18]:
print(str(eval_result.passing))

True


In [22]:
# query index
query_engine = index.as_query_engine()
response = query_engine.query(
    "What is the Capital os India?"
)

In [23]:
eval_result = evaluator.evaluate_response(response=response)

In [24]:
print(str(eval_result.passing))

False


In [25]:
# query index
query_engine = index.as_query_engine()
response = query_engine.query(
    "What is the Economic Scenario Indian Economy?"
)
eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.passing))

True


In [26]:
# query index

query = "What is the Economic Scenario Indian Economy?"
response = query_engine.query(query)
eval_result = evaluator.evaluate_response(query=query, response=response)
print(str(eval_result))

query='What is the Economic Scenario Indian Economy?' contexts=['Indian Economy: An Overview\nCurrent Economic Scenario\nAs of 2024, the Indian economy continues to grow, albeit at a slower pace than in previous decades.\nThe COVID-19 pandemic had a profound impact, causing a contraction in GDP in 2020-21.\nHowever, a robust recovery is underway, driven by government stimulus measures, digitalization,\nand strong domestic consumption.\nInflation remains a challenge, driven by supply chain disruptions and rising commodity prices. The\ngovernment has focused on infrastructure development, digital economy, and social welfare\nschemes to drive growth. Initiatives such as the Goods and Services Tax (GST) and Insolvency and\nBankruptcy Code (IBC) have been instrumental in improving the business environment.\nPage 6', "Indian Economy: An Overview\nIntroduction\nIndia, one of the world's fastest-growing economies, has shown remarkable resilience and growth\nover the past few decades. With a po

In [28]:
# query index

query = "What is the Capital of USA?"
response = query_engine.query(query)
eval_result = evaluator.evaluate_response(query=query, response=response)
print(str(eval_result))

query='What is the Capital of USA?' contexts=['Indian Economy: An Overview\nIndian Economy: An Overview\nPage 1', "Indian Economy: An Overview\nConclusion\nThe Indian economy has come a long way since independence, evolving into a dynamic and diverse\neconomic powerhouse. While significant challenges remain, the country's resilience, young\npopulation, and ongoing reforms provide a strong foundation for future growth. By addressing its\nstructural issues and leveraging its strengths, India is poised to become a major player in the global\neconomy.\nPage 9"] response='Washington, D.C.' passing=False feedback='NO' score=0.0 pairwise_source=None invalid_result=False invalid_reason=None


In [30]:
# query index
query_engine = index.as_query_engine()
query = "What are the Key Sectors of the Indian Economy?"
response = query_engine.query(query)
response_str = response.response
for source_node in response.source_nodes:
    eval_result = evaluator.evaluate(
        query=query,
        response=response_str,
        contexts=[source_node.get_content()],
    )
    print(str(eval_result.passing))

True
True


Question Generation

In [31]:
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

In [32]:
# create llm
llm = OpenAI(model="gpt-4", temperature=0.0)

# build documents
documents = SimpleDirectoryReader("/content").load_data()


In [33]:
# define generator, generate questions
dataset_generator = RagDatasetGenerator.from_documents(
    documents=documents,
    llm=llm,
    num_questions_per_chunk=10,  # set the number of questions per nodes
)

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1478: RuntimeWarning: coroutine 'FaithfulnessEvaluator.aevaluate' was never awaited
  _C._initExtension(manager_path())


In [34]:
rag_dataset = dataset_generator.generate_questions_from_nodes()
questions = [e.query for e in rag_dataset.examples]

In [35]:
print(questions)

['What is the title of the document mentioned in the context?', 'What is the file path of the document?', 'What is the main topic of the document?', 'What is the label of the page mentioned in the context?', 'What is the subject of the document based on the context provided?', 'What could be the possible content of the document based on its title?', 'What is the page number of the document mentioned in the context?', 'Based on the context, what could be the possible subtopics in the document?', 'What type of document do you think this is, based on the context provided?', 'Can you guess the possible structure of the document based on the context?', 'What is the current population of India and where does it rank globally in terms of population?', 'What are the three main sectors that encompass the Indian economy?', 'What has been the growth trend of the Indian economy over the past few decades?', "What is the significance of India being one of the world's fastest-growing economies?", 'Ca

Batch Evaluation

In [41]:
from llama_index.core.evaluation import RelevancyEvaluator
relevancy_evaluator = RelevancyEvaluator()

In [43]:
from llama_index.core.evaluation import BatchEvalRunner

runner = BatchEvalRunner(
    {"faithfulness": evaluator, "relevancy": relevancy_evaluator},
    workers=8,
)

eval_results = await runner.aevaluate_queries(
    index.as_query_engine(), queries=questions
)

In [45]:
print(str(eval_results))

{'faithfulness': [EvaluationResult(query='What is the title of the document mentioned in the context?', contexts=['Indian Economy: An Overview\nIndian Economy: An Overview\nPage 1', 'Indian Economy: An Overview\nFuture Prospects\nThe future prospects of the Indian economy remain positive, driven by several factors:\n- Demographic Dividend: A young and growing population provides a vast labor force and consumer\nmarket.\n- Technological Advancements: Rapid digitalization and technological adoption are transforming\nvarious sectors, enhancing productivity and innovation.\n- Policy Reforms: Continued focus on economic reforms, ease of doing business, and infrastructure\ndevelopment will support sustainable growth.\n- Global Integration: Increasing integration with the global economy through trade agreements and\nFDI will open new avenues for growth.\nHowever, to fully realize its potential, India must address structural issues, invest in human capital,\nand ensure inclusive growth.\nPage 